In [1]:
import pandas as pd
from datetime import date

In [2]:
import time

In [81]:
df = pd.read_csv(r'Data/scopus_bd_aumentada.csv', sep=",")

C:\Users\YORLEY\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7,21,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [115]:
df_authors = df[['Authors_Rev','Author(s)_ID_Rev']][:].drop_duplicates(subset='Author(s)_ID_Rev')

### Total Citaciones

In [116]:
df_temp=df[['Author(s)_ID_Rev', 'Cited_by']].groupby(by='Author(s)_ID_Rev').sum()
df_temp.reset_index(level=0, inplace=True)

In [117]:
df_authors = df_authors.merge(right=df_temp, how='inner', on='Author(s)_ID_Rev')

### Total Artículos publicados

In [118]:
df_temp=df[['Author(s)_ID_Rev', 'Title']].groupby(by='Author(s)_ID_Rev').count()
#Convierte el indice en otro campo
df_temp.reset_index(level=0, inplace=True)

In [119]:
df_authors = df_authors.merge(df_temp, how='inner', on ='Author(s)_ID_Rev')

### Máximo número de citaciones

In [120]:
df_temp=df[['Author(s)_ID_Rev', 'Cited_by']].groupby(by='Author(s)_ID_Rev').max()
df_temp.reset_index(level=0, inplace=True)

In [121]:
df_authors = df_authors.merge(right=df_temp, how='inner', on='Author(s)_ID_Rev')
df_authors.columns = ['Authors_Rev', 'Author(s)_ID_Rev', 'Total_Citation', 'Total_Title', 'Max_Citation']
df_authors['Max_Citation'].fillna(0, inplace=True)

### Número de Revistas

In [122]:
df_temp=df[['Author(s)_ID_Rev', 'Source_title']].groupby(by=['Author(s)_ID_Rev']).nunique()
df_temp = df_temp[['Source_title']]
df_temp.reset_index(level=0, inplace=True)
df_temp.columns = ['Author(s)_ID_Rev','Total_Journal']
df_temp.sort_values(by='Total_Journal', ascending = False, inplace= True)

In [123]:
df_authors = df_authors.merge(df_temp, how='inner', on ='Author(s)_ID_Rev')

In [124]:
año_actual=int(date.today().strftime("%Y"))
año_pasado=año_actual-1

In [125]:
df2_años=df[(df['Year']==año_actual) | (df['Year']==año_pasado) ]

### Total Publicaciones últimos dos años

In [126]:
df_temp=df2_años[['Author(s)_ID_Rev', 'Title']].groupby(by='Author(s)_ID_Rev').count()
df_temp.reset_index(level=0, inplace=True)
df_temp.columns = ['Author(s)_ID_Rev','Delta_publications']
df_authors = df_authors.merge(df_temp, how='left', on ='Author(s)_ID_Rev')

In [127]:
df_authors['Delta_publications'].fillna(0, inplace=True)


### Carrera Autor

In [128]:
df_temp = df[['Author(s)_ID_Rev','Year']].groupby(by = 'Author(s)_ID_Rev').min()
df_temp.reset_index(level=0, inplace=True)
df2_temp = df[['Author(s)_ID_Rev','Year']].groupby(by = 'Author(s)_ID_Rev').max()
df2_temp.reset_index(level=0, inplace=True)
df_temp = df_temp.merge(df2_temp, how='left', on='Author(s)_ID_Rev')
df_temp['autor_career'] =  df_temp['Year_y'] - df_temp['Year_x']
df_authors = df_authors.merge(df_temp[['Author(s)_ID_Rev','autor_career']], how='left', on ='Author(s)_ID_Rev')

### Citación Media por Revista

In [129]:
df_temp = df[['Source_title','Cited_by']].groupby(by='Source_title').mean().fillna(0)
df_temp.reset_index(level=0, inplace=True)

In [130]:
df2_temp = df[['Author(s)_ID_Rev','Source_title','Cited_by']].groupby(by=['Author(s)_ID_Rev','Source_title']).mean().fillna(0)
df2_temp.reset_index(level=['Author(s)_ID_Rev','Source_title'],inplace=True)
df2_temp =df2_temp.merge(df_temp, how='left', on='Source_title')
df2_temp = df2_temp[['Author(s)_ID_Rev','Cited_by_y']].groupby(by='Author(s)_ID_Rev').mean()
df2_temp.reset_index(level=0,inplace=True)
df2_temp.columns = ['Author(s)_ID_Rev','citation_journal']
df_authors = df_authors.merge(df2_temp, how='left', on='Author(s)_ID_Rev')

### Total Coautores ultimos dos años

#### Leer
* Identificación de Todos los autores
* para cada autor se revisan los papers que en los que aparece
* para cada paper de ese autor se revisan cuantas veces aparece bajo un nombre diferente al del autor superior
* si el autor que aparece no se ha añadido a la lista de coautores se hace.

# Esto se demora corriendo alrededor de 10 minutos
## puedes irte a tomar una bebida caliente mientras tanto
### es en serio :( :(

In [103]:
start = time.time()
#Línea de prueba para correr rápido
#lista_autores = df2_años['Author(s)_ID_Rev'][df2_años['Author(s)_ID_Rev']=='53866250100'].unique()
lista_autores = df2_años['Author(s)_ID_Rev'].unique()[:]
lista_coautores = []
for autor in lista_autores:
    df_paper = df2_años[['Author(s)_ID_Rev','Title']][df2_años['Author(s)_ID_Rev']==autor]
    lista_papers = df_paper['Title']
    coautores = []
    for paper in lista_papers:
        coautor = (df2_años[['Author(s)_ID_Rev','Title']][(df2_años['Title']==paper)&(df2_años['Author(s)_ID_Rev']!=autor)]
                   ['Author(s)_ID_Rev'])
        if len(coautor)==0:
            continue
        else:
            coautor = (df2_años[['Author(s)_ID_Rev','Title']][(df2_años['Title']==paper)&(df2_años['Author(s)_ID_Rev']!=autor)]
                   ['Author(s)_ID_Rev'])
        for c in coautor:
            if c is not coautores:
                coautores.append(c)
    del(df_paper)
    lista_coautores.append([autor,len(coautores)])
end = time.time()
#print((end - start)/60)

In [104]:
#Tiempo de ejecución
(end -start)/60

9.296003353595733

In [113]:
#lista_coautores

In [131]:
df2_temp = pd.DataFrame(lista_coautores, columns=['Author(s)_ID_Rev','Delta_coautores'])


In [132]:
df_authors = df_authors.merge(df2_temp, how='left', on='Author(s)_ID_Rev').sort_values(by='Delta_publications', ascending = False)

# Indicadores Papers

* Citaciones por Artículos
* años del artículos
* Cantidad de coautures

In [108]:
df_papers =df[['Title','Cited_by']].drop_duplicates(subset='Title').fillna(0)
df_temp = df[['Title','Year']].drop_duplicates(subset='Title').fillna(0)
df_temp['Year'] = año_actual - df_temp['Year']
df_papers = df_papers.merge(df_temp, how='left', on = 'Title')
df_temp = df[['Title', 'Author(s)_ID_Rev']].fillna(0).groupby(by='Title').count()
df_temp.reset_index(level=0, inplace= True)
df_papers = df_papers.merge(df_temp, how='left', on = 'Title')
df_papers.columns = ['Title', 'Cited_by', 'Years', 'cant_coautores']

### Citaciones por revista

In [109]:
#Revistas y Citaciones
df_temp = df[['Source_title','Cited_by']].groupby(by='Source_title').mean().fillna(0)
df_temp.reset_index(level=0, inplace=True)

In [110]:
df2_temp = df[['Title','Source_title','Cited_by']].groupby(by=['Title','Source_title']).mean().fillna(0)
df2_temp.reset_index(level=['Title','Source_title'],inplace=True)
df2_temp =df2_temp.merge(df_temp, how='left', on='Source_title')
df2_temp = df2_temp[['Title','Cited_by_y']].groupby(by='Title').mean()
df2_temp.reset_index(level=0,inplace=True)
df2_temp.columns = ['Title','citation_journal']
df_papers = df_papers.merge(df2_temp, how='left', on='Title')

In [133]:
df_authors.to_csv(r'Data/indicadores_autores.csv')
df_papers.to_csv(r'Data/indicadores_papers.csv')